In [ ]:
import pandas as pd
data=pd.read_feather('sample.feather')
pd.options.display.max_columns = 700
data

In [2]:
import numpy as np
df=data
X = df.drop(['Treatment'],axis = 1)
y = df.Treatment
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.dropna(inplace=True)
#X.fillna(0,inplace=True)
X_numerical = X.select_dtypes('number')
X_useful=X_numerical.loc[:,X_numerical.iloc[1:5,].nunique() > 1]
X_useful=X_useful.drop(['ObjectNumber','Number_Object_Number','Parent_PrefilterCellsed','Parent_Cells'],axis=1,errors='ignore')
df_preprocessed=X_useful.join(y)

In [3]:
df_NT=df_preprocessed.loc[df_preprocessed.Treatment=='a',df_preprocessed.columns!='Treatment']
df_TNT=df_preprocessed.loc[df_preprocessed.Treatment=='b',df_preprocessed.columns!='Treatment']
df_preprocessed.loc[len(df_preprocessed)]=np.abs(df_NT.mean() - df_TNT.mean()) / np.sqrt(np.square(df_NT.std())+np.square(df_TNT.std()))
df_preprocessed.loc[len(df_preprocessed)]=1 - (3*(df_NT.std()+df_TNT.std())/np.abs(df_NT.mean() - df_TNT.mean()))

In [ ]:
df_preprocessed

In [ ]:
import pandas as pd
import numpy as np
df_ssmd=df_preprocessed.loc[len(df_preprocessed)-2:len(df_preprocessed)-1,df_preprocessed.columns!='Treatment']
df_zprime=df_preprocessed.loc[len(df_preprocessed)-1:,df_preprocessed.columns!='Treatment']
# Finding the column with the largest value
max_column_index = np.argmax(df_ssmd.values)
max_column = df_ssmd.columns[max_column_index % len(df_ssmd.columns)]
print(f"The column with the largest value is: {max_column}")



In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'xx-large',
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
pylab.rcParams.update(params)
plt.figure(figsize=(6,6))
df_ssmd.sort_values(by=269140,axis=1,ascending=False).iloc[:1,:10].transpose().plot(kind='barh')
plt.xlim(0.8,1)
plt.legend(labels=['ssmd'])

In [ ]:
plt.figure(figsize=(6,6))
df_zprime.sort_values(by=269141,axis=1,ascending=False).iloc[:1,:10].transpose().plot(kind='barh')
plt.legend(labels=['zprime'])


In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline
import umap

reducer = umap.UMAP()
df_sample=df_preprocessed.groupby("Treatment").sample(n=1000, random_state=1)
df_UMAP = df_sample.loc[:,df_sample.columns!='Treatment']
scaled_df_UMAP = StandardScaler().fit_transform(df_UMAP)
embedding = reducer.fit_transform(scaled_df_UMAP)
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=[sns.color_palette()[x] for x in df_sample.Treatment.map({"NT":0, "TNT":1, "TGFB1":2,"TNF-alpha":3})])
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the dataset')

In [18]:
!pip install umap-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.7 MB/s eta 0:00:00
